# User Guide

> Instructions for using `nbstata` as a Jupyter kernel

## Install

*Because it uses [pystata](https://www.stata.com/python/pystata/) under the hood, `nbstata` requires Stata 17 to be installed locally.
(If you have an older version of Stata, consider [stata_kernel](https://github.com/kylebarron/stata_kernel) instead.)*

To install `nbstata`, run:

```sh
pip install nbstata
python -m nbstata.install [--sys-prefix] [--prefix PREFIX] [--conf-file]
```

Include `--sys-prefix` to install to `sys.prefix` (e.g. a virtualenv or conda env),
or `--prefix PREFIX` if you want to specify the install path yourself.

#### Configuration file
The `--conf-file` option creates a configuration file for you.
(Note: If the installer cannot find the location of your Stata installation, a configuration file will be created 
even if you do not include the `--conf-file` option to allow you to manually specify the Stata location.) The location of the configuration file will be:

- `[prefix]/etc/nbstata.conf` if `--sys-prefix` or `--prefix` is specified.
-  `~/.nbstata.conf` otherwise.

(Note: If a configuration file exists in both locations at kernel runtime, the user version takes precedence.)

#### Updating

To update from a previous version of `nbstata`, run:

```sh
pip install nbstata --upgrade
```

When updating, you don't have to run `python -m nbstata.install` again.

### Configuration

The following settings are permitted inside the configuration file:

- `stata_dir`: Stata installation directory.
- `edition`: Stata edition. Acceptable values are 'be', 'se' and 'mp'.
    Default is 'be'.
- `graph_format`: Acceptable values are 'png' (the default), 'pdf', 'svg' and 'pystata'.
    Specify the last option if you want to use `pystata`'s [default setting](https://www.stata.com/python/pystata/config.html#pystata.config.set_graph_format). 
- `echo`: controls the echo of commands, with the default being 'None':
    - 'True': the kernel will echo all commands. 
    - 'False': the kernel will not echo single-line commands.
    - 'None': the kernel will not echo any command. 
- `splash`: controls display of the splash message during Stata startup. Default is 'False'.
- `missing`: What should be displayed in the output of the `*%head` and `*%tail` magics for a missing value. Default is '.', following Stata. To defer to pandas' format for `NA`, specify 'pandas'.

Settings must be under the title `[nbstata]`. Example:

```
[nbstata]
stata_dir = /opt/stata
edition = mp
graph_format = svg
echo = False
splash = True
missing = NA
```

#### Default Graph Format

Both `pystata` and `stata_kernel` default to the SVG image format. 
`nbstata` defaults to the PNG image format instead for several reasons:

- Jupyter does not show SVG images from untrusted notebooks ([link 1](https://stackoverflow.com/questions/68398033/svg-figures-hidden-in-jupyterlab-after-some-time)).
- Notebooks with empty cells are untrusted ([link 2](https://github.com/jupyterlab/jupyterlab/issues/9765)).
- SVG images cannot be copied and pasted directly into Word or PowerPoint.

### Syntax highlighting
Stata syntax highlighting can be installed for Jupyter Lab:

```sh
pip install jupyterlab_stata_highlight2
```

(If you prefer the standard Jupyter color scheme, the original [jupyterlab-stata-highlight](https://kylebarron.dev/stata_kernel/using_jupyter/lab/#syntax-highlighting) also works.)

## Magics

'Magics' are commands provided by `nbstata` that enhance the experience of working with Stata in Jupyter. They work only when placed at the beginning of a code cell. 

Jupyter magics typically start with `%`, but `nbstata` magics may alternatively be prefixed with `*%` so that, if you export a Stata notebook to a .do file and run it that way, the magics will not cause errors.

`nbstata` currently supports the following magics:

| Magic | Description | Full Syntax |
| :-- | :-- | :-- |
| %browse | Interactively view dataset | `%browse [-h] [varlist] [if] [in] [, nolabel noformat]` |
| %head | View first 5 (or N) rows | `%head [-h] [N] [varlist] [if] [, nolabel noformat]` |
| %tail | View last 5 (or N) rows | `%tail [-h] [N] [varlist] [if] [, nolabel noformat]` |
| %locals | List locals with their values | `%locals` |
| %delimit | Print the current delimiter | `%delimit` |
| %help | Display Stata help | `%help [-h] command_or_topic_name` |
| %set | Set single config option | `%set [-h] key = value` |
| %%set | Set multiple config options | `%%set [-h]` |
| %%echo | Ensure echo from cell | `%%echo` |
| %%noecho | Suppress echo from cell | `%%noecho` |
| %%quietly | Suppress all output from cell | `%%quietly` |

You can run any magic with the `-h` option (`--help`) to see brief help documentation for the magic.

### `%browse`, `%head`, `%tail`
*Quickly view your data*

```
*%browse [-h] [varlist] [if] [in] [, nolabel noformat]
*%head [-h] [N] [varlist] [if] [, nolabel noformat]
*%tail [-h] [N] [varlist] [if] [, nolabel noformat]
```

These magics provide alternatives to Stata's `browse` command, which is not available in a Stata notebook. They can each be called with standard Stata `varlist` and `if` syntax. `%browse` also supports Stata's `in` syntax, whereas `%head` (and `%tail`), modeled after [pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), display the first (or last) 5 (or `N`) observations that meet the (optional) `if` criteria.

By default, the `%browse`, `%head`, and `%tail` magics convert numeric Stata values to strings 
using their Stata format and value labels. To prevent this behavior, 
specify the `noformat` and/or `nolabel` options.

#### `%browse` installation
The `%head` and `%tail` work out of the box, but `%browse` requires [prior installation](https://perspective.finos.org/docs/python/#jupyterlab) of the `@finos/perspective-jupyterlab` JupyterLab extension.

### `%locals`
*List local macro names and values*

This takes no arguments. The output format mimics Stata's `macro list` command (which only displays global macros).

### `%delimit`

*Print the current Stata command delimiter*

This takes no arguments; it prints the delimiter currently set: either `cr` or
`;`. If you want to change the delimiter, use `#delimit ;` or `#delimit cr`. The
delimiter will remain set until changed.

```
[1]: %delimit
Current Stata command delimiter: cr
[2]: #delimit ;
delimiter now ;
[3]: *%delimit
Current Stata command delimiter: ;
[4]: #delimit cr
delimiter now cr
```

### `%help`

*Display a help file in rich text*

```
*%help [-h] command_or_topic_name
```

Add the term you want to search for after `%help`. For example:

![Jupyter Notebook help example](https://github.com/kylebarron/stata_kernel/raw/master/docs/src/img/notebook_help_magic.png)

The underlined terms in the output are _links_. Click
on them to open further help in a new tab.

### `%set`, `%%set`

*Set configuration values*

Usage:
```
*%set [-h] key = value
```
```
*%%set
key1 = value1
[key2 = value2]
[...]
```

- `key`: Configuration key name: `graph_format`, `echo`, or `missing`
- `value`: Value to set. See [Configuration](#configuration) above for more information.

Examples:

```
*%set graph_format = svg
```
```
%%set
echo = True
missing = N/A
```
To prevent the cell magic `%%set` from causing an error if you export the notebook to a .do file and run it that way, you may surround the key-value statements with `/*` and `*/` on separate lines, like this:
```
*%%set
/*
echo = True
missing = N/A
*/
```

### `%%echo` `%%noecho`, `%%quietly`
*Toggle cell output type*

Putting `%%echo` at the top of a cell sets the configuration option `echo = True` *for just that cell*. For example, suppose you have configured `echo = None` but you do want to see the Stata commands echoed for a particular cell:
```
[1]: *%%echo
     disp 1
     disp 2
. disp 1
1

. disp 2
2

. 

```

Similarly, `%%noecho` sets the configuration option `echo = None` for a single cell:
```
[2]: *%%noecho
     disp 1
     disp 2
1
2
```

`%%quietly` silences all cell output, including graphs. It is a convenience magic equivalent to placing the standard Stata code `quietly {` at the start and `}` at the end of the cell.
```
[3]: *%%quietly
     disp 1
     disp 2
```

## Stata Implementation Details

#### `#delimit` behavior

A [`#delimit;`](https://www.stata.com/manuals/pdelimit.pdf) command in one cell will persist into other cells, until `#delimit cr` is called. For example, see [delimit tests.ipynb](https://github.com/hugetim/nbstata/blob/master/manual_test_nbs/delimit%20tests.ipynb).

#### `echo = None`: potential for unanticipated errors

The default `echo = None` configuration does some complicated things under the hood to emulate functionality that `pystata` does not directly support: running multi-line Stata code without echoing the commands. While extensive automatic tests are in place to help ensure its reliability, unanticipated issues may arise. If, while using this mode, a particular code cell is not working as expected, try placing the `%%echo` magic at the top of it to see if that resolves the issue. (If so, please report that [here](https://github.com/hugetim/nbstata/issues/new?labels=bug).) You can also avoid such potential issues by setting the config `echo = False`, which will at least not echo single-line Stata commands though it will echo multiple commands.

#### `more` and `pause`

Stata's [more](https://www.stata.com/help.cgi?more) and [pause](https://www.stata.com/help.cgi?pause) commands do not work in a notebook, so these features should remain in their default 'off' states (i.e., `set more off` and `pause off`).